# Tune a forecasting model using DeepAR on SageMaker

## Setup environment

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

region = sagemaker.Session().boto_region_name
role = sagemaker.get_execution_role()
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, 'forecasting-deepar', 'latest')

## Define data inputs from S3

In [ ]:
# Replace with your S3 dataset path
inputs = {
    'train': '<your-train-data-s3-path>',
    'test': '<your-test-data-s3-path>'
}
print(inputs)

## Tune

In [ ]:
freq = 'H'
prediction_length = 48
context_length = 72

hyperparameters = {
    "time_freq": freq,
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_cells": "40",
    "num_layers": "3",
    "likelihood": "gaussian",
    "mini_batch_size": "32",
    "learning_rate": "0.001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10"
}

In [ ]:
estimator = Estimator(
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c5.xlarge',
    train_volume_size=50,
    hyperparameters=hyperparameters,
    base_job_name='deepar-electricity'
)

hyperparameter_ranges = {
        'epochs': IntegerParameter(10, 20),
    }

objective_metric_name = 'test:RMSE'
objective_type = 'Minimize'

In [ ]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=3,
                            max_parallel_jobs=1,
                            objective_type=objective_type)

tuner.fit(inputs)

In [ ]:
print(tuner.best_training_job())